<a href="https://colab.research.google.com/github/VGGatGitHub/Summer2021/blob/main/QISKit_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook experimenting how to use DOcplex and IBM Quantum.

# 0. Install libraries & clone repo

In [ ]:
import sys

if hasattr(sys, 'real_prefix'):
    #we are in a virtual env.
    !pip install cplex && pip install docplex
else:
    !pip install --user cplex && pip install --user docplex

print()
!pip show cplex
print()
!pip show docplex

#it my be useful to retstart you runtime

In [ ]:
!pip install qiskit
!pip show qiskit

# !pip install qiskit-optimization
#!pip show qiskit-optimization

#it my be useful to retstart you runtime

**Possibly need to restart runtime here.**

In [3]:
#it my be useful to retstart you runtime

# clone repo
!git clone https://github.com/VGGatGitHub/Summer2021.git

Cloning into 'Summer2021'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (98/98), done.
remote: Total 112 (delta 52), reused 19 (delta 10), pack-reused 0
Receiving objects: 100% (112/112), 205.01 KiB | 6.41 MiB/s, done.
Resolving deltas: 100% (52/52), done.


# 1. Load Data

In [1]:
#next run the model.py in the folder introScenario
!pwd
%cd ./Summer2021/introScenario/
#./Summer2021/intermediateScenario
#./Summer2021/introScenario/
#%ls


/content
/content/Summer2021/introScenario


In [13]:
import pandas as pd

# supply + costs
supply_file = './plants.csv'

# demand
demand_file = './customerDemand.csv'

# for now load example data (plants.csv and customerDemand.csv under introScenario folder)
s_df = pd.read_csv(supply_file)
d_df = pd.read_csv(demand_file)

In [14]:
d_df

,Product,Demand
0,handSanitizer,100
1,mask,120


In [15]:
s_df

,Plants,Cost,Capacity,Product
0,1,3,40,mask
1,2,2,30,mask
2,3,1,30,handSanitizer
3,4,3,100,handSanitizer
4,5,2,60,mask
5,6,1,45,mask


In [16]:
s_df.drop([1,2,3], inplace = True)
s_df

,Plants,Cost,Capacity,Product
0,1,3,40,mask
4,5,2,60,mask
5,6,1,45,mask


In [17]:
inputs={'customerDemand':d_df,'plants':s_df}

# 2. Solve the DO problem via DOcplex

In [18]:
#If you get an error you may have to restart the runtime
import cplex
import docplex.mp
from docplex.mp.model import Model

In [19]:
global output_lock
#with output_lock:

try:  
  if output_lock:
    print(output_lock)
except:
  output_lock=dict({'output_lock':{}})

try: 
  if outputs:
    print(outputs)
except:
  outputs=dict()


In [20]:
exec(open("./model.py").read())

* building wado model
model building done in 0.0822141170501709 secs
* running wado model
Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               120
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 10 rows and 6 columns.
All rows and columns eliminated.
Presolve time = 0.01 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.02 sec. (0.01 ticks)
Parallel b&c, 2 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.02 sec. (0.01 ticks)
Solve status: JobSolveStatus.OPTIMAL_SOLUTION
* model docplex_model1 solved with objective = -58080.000
*  KPI: total plants allocations                  = 61440.000
*  KPI: total Cost of plants over all allocat

In [21]:
outputs


{'plantsAllocation':    plants  ...  plants minValueAllocationForAssignment
 0       1  ...                                       0
 1       5  ...                                       0
 2       6  ...                                       0
 
 [3 rows x 7 columns]}

In [22]:
outputs['plantsAllocation']

,plants,plants allocation decision,plants selection decision,plants Capacity,plants Product,plants Cost,plants minValueAllocationForAssignment
0,1,15.0,1.0,40,mask,3,0
1,5,60.0,1.0,60,mask,2,0
2,6,45.0,1.0,45,mask,1,0


In [26]:
model

docplex.mp.Model['docplex_model1']

In [27]:
solution=model.solve()
print(solution)
solution

solution for: docplex_model1
objective: -58080
x1=15
x2=60
x3=45
x4=1
x5=1
x6=1



docplex.mp.solution.SolveSolution(obj=-58080,values={x1:15,x2:60,x3:45,x..

In [28]:
cost = solution.get_objective_value()
x = solution.get_all_values()
x = np.array(x)
print(cost,x)

-58080.0 [15. 60. 45.  1.  1.  1.]


#Minimum Eigen Optimizers: exact, qaoa, rqaoa

In [35]:
#https://quantum-computing.ibm.com/lab/docs/iql/optimization

from qiskit import BasicAer
from qiskit.aqua import aqua_globals, QuantumInstance
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram

aqua_globals.random_seed = 10598
#('qasm_simulator') #qasm_simulator or statevector_simulator

quantum_instance = QuantumInstance(BasicAer.get_backend('qasm_simulator'),
                                   seed_simulator=aqua_globals.random_seed,
                                   seed_transpiler=aqua_globals.random_seed)
qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0., 0.])
exact_mes = NumPyMinimumEigensolver()

qaoa = MinimumEigenOptimizer(qaoa_mes)   # using QAOA
exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver
rqaoa = RecursiveMinimumEigenOptimizer(min_eigen_optimizer=qaoa, min_num_vars=1, min_num_vars_optimizer=exact)


In [ ]:
# create a QUBO
if mod:
  qubo=mod
  print(mod.export_as_lp_string())
else:
  qubo = QuadraticProgram()
  qubo.binary_var('x')
  qubo.binary_var('y')
  qubo.binary_var('z')
  qubo.minimize(linear=[1,-2,3], quadratic={('x', 'y'): 1, ('x', 'z'): -1, ('y', 'z'): 2})
  print(qubo.export_as_lp_string())


In [ ]:
%time rqaoa_result = rqaoa.solve(qubo)
print(rqaoa_result)

In [ ]:
%time qaoa_result = qaoa.solve(qubo)
print(qaoa_result)

In [ ]:
%time exact_result = exact.solve(qubo)
print(exact_result)

In [ ]:
op, offset = qubo.to_ising()
qp=QuadraticProgram()
qp.from_ising(op, offset, linear=True)

In [ ]:
%time exact_result = exact.solve(qp)
print(exact_result)

#Try Grover Optimizer: grover_optimizer

In [115]:
from qiskit.aqua.algorithms import NumPyMinimumEigensolver
from qiskit.optimization.algorithms import GroverOptimizer, MinimumEigenOptimizer
from qiskit.optimization.problems import QuadraticProgram
from qiskit import BasicAer
from docplex.mp.model import Model

backend = BasicAer.get_backend('statevector_simulator') #qasm_simulator or statevector_simulator
grover_optimizer = GroverOptimizer(6, num_iterations=10, quantum_instance=backend)


In [116]:
%time results = grover_optimizer.solve(qp)#qubo or qp
print("x={}".format(results.x))
print("fval={}".format(results.fval))

CPU times: user 54.8 s, sys: 2.29 s, total: 57.1 s
Wall time: 53.1 s
x=[0. 1. 0.]
fval=-2.0


In [ ]:
%time results = grover_optimizer.solve(qubo)#qubo or qp
print("x={}".format(results.x))
print("fval={}".format(results.fval))

In [ ]:
%time results = grover_optimizer.solve(qubo)#qubo or qp
print("x={}".format(results.x))
print("fval={}".format(results.fval))

#Try ADMM Optimizer (using cobyla or cplex)


In [118]:
import time
from typing import List, Optional, Any
import numpy as np
import matplotlib.pyplot as plt 

from docplex.mp.model import Model

from qiskit import BasicAer
from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import CobylaOptimizer, MinimumEigenOptimizer
from qiskit.optimization.problems import QuadraticProgram
from qiskit.optimization.algorithms.admm_optimizer import ADMMParameters, ADMMOptimizer

# If CPLEX is installed, you can uncomment this line to import the CplexOptimizer.
# CPLEX can be used in this tutorial to solve the convex continuous problem, 
# but also as a reference to solve the QUBO, or even the full problem.
# 
from qiskit.optimization.algorithms import CplexOptimizer

In [119]:
# define COBYLA optimizer to handle convex continuous problems.
cobyla = CobylaOptimizer()

# define QAOA via the minimum eigen optimizer
qaoa = MinimumEigenOptimizer(QAOA(quantum_instance=BasicAer.get_backend('qasm_simulator')))
#('qasm_simulator') #qasm_simulator or statevector_simulator
# exact QUBO solver as classical benchmark
exact = MinimumEigenOptimizer(NumPyMinimumEigensolver()) # to solve QUBOs


In [120]:
print(qubo)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: x - 2 y + 3 z + [ 2 x*y - 2 x*z + 4 y*z ]/2
Subject To

Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 1

Binaries
 x y z
End



In [121]:
admm_params = ADMMParameters(
                            rho_initial=1001, 
                            beta=1000, 
                            factor_c=900,
                            maxiter=100,
                            three_block=True, tol=1.e-6
                        )

In [ ]:
# in case CPLEX is installed it can also be used for the convex problems, the QUBO, 
# or as a benchmark for the full problem.
#
cplex = CplexOptimizer()

# define QUBO optimizer
# qubo_optimizer = exact #qaoa #rqaoa #grover_optimizer

# qubo_optimizer = cplex  # uncomment to use CPLEX instead

i=-1
optms=['cplex','exact','qaoa','rqaoa']
for qubo_optimizer in [cplex,exact,qaoa,rqaoa]:

  # define classical optimizer
  convex_optimizer = cobyla
  #convex_optimizer = cplex  # uncomment to use CPLEX instead

  # initialize ADMM with classical QUBO and convex optimizer
  admm = ADMMOptimizer(params=admm_params, 
                      qubo_optimizer=qubo_optimizer,
                      continuous_optimizer=convex_optimizer)

  # run ADMM to solve problem
  i+=1
  print("\nOptimizer:",optms[i])
  %time result = admm.solve(mod)
  print("x={}".format(result.x))
  print("fval={:.2f}".format(result.fval))

In [ ]:
# run ADMM to solve problem
%time result = admm.solve(qp)
print("x={}".format(result.x))
print("fval={:.2f}".format(result.fval))

In [ ]:
%time result = admm.solve(qubo)
print("x={}".format(result.x))
print("fval={:.2f}".format(result.fval))

# 3. Solve the problem using QISKit

In [29]:
# take a look at https://quantum-computing.ibm.com/lab/docs/iql/optimization

#from qiskit_optimization import QuadraticProgram
#from qiskit_optimization.translators import from_docplex_mp

from qiskit.optimization import QuadraticProgram

# load from a Docplex model
mod = QuadraticProgram()
mod.from_docplex(model)
print(mod.export_as_lp_string())

/usr/local/lib/python3.7/dist-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
/usr/local/lib/python3.7/dist-packages/qiskit/optimization/__init__.py:92: DeprecationWarning: The package qiskit.optimization is deprecated. It was moved/refactored to qiskit_optimization (pip install qiskit-optimization). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('optimization', 'qiskit_optimization', 'qiskit-optimization')


\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model1

Minimize
 obj: - 464 x0 - 480 x1 - 496 x2
Subject To
 _L_EXPR_1: x0 - 40 x3 <= 0
 _L_EXPR_2: x1 - 60 x4 <= 0
 _L_EXPR_3: x2 - 45 x5 <= 0
 _L_EXPR_4: x0 >= 0
 _L_EXPR_5: x1 >= 0
 _L_EXPR_6: x2 >= 0
 _L_EXPR_7: x0 <= 40
 _L_EXPR_8: x1 <= 60
 _L_EXPR_9: x2 <= 45
 _L_EXPR_10: x0 + x1 + x2 <= 120

Bounds
 0 <= x3 <= 1
 0 <= x4 <= 1
 0 <= x5 <= 1

Binaries
 x3 x4 x5

Generals
 x0 x1 x2
End



In [125]:
print('constant:\t\t\t', mod.objective.constant)

print('linear dict:\t\t\t', mod.objective.linear.to_dict())
print('linear array:\t\t\t', mod.objective.linear.to_array())
print('linear array as sparse matrix:\n', mod.objective.linear.coefficients, '\n')

print('quadratic dict w/ index:\t', mod.objective.quadratic.to_dict())
print('quadratic dict w/ name:\t\t', mod.objective.quadratic.to_dict(use_name=True))
print('symmetric quadratic dict w/ name:\t', mod.objective.quadratic.to_dict(use_name=True, symmetric=True))
print('quadratic matrix:\n', mod.objective.quadratic.to_array(),'\n')
print('symmetric quadratic matrix:\n', mod.objective.quadratic.to_array(symmetric=True),'\n')
print('quadratic matrix as sparse matrix:\n', mod.objective.quadratic.coefficients)


constant:			 0.0
linear dict:			 {0: -464.0, 1: -480.0, 2: -480.0, 3: -496.0}
linear array:			 [-464. -480. -480. -496.    0.    0.    0.    0.]
linear array as sparse matrix:
   (0, 0)	-464.0
  (0, 1)	-480.0
  (0, 2)	-480.0
  (0, 3)	-496.0 

quadratic dict w/ index:	 {}
quadratic dict w/ name:		 {}
symmetric quadratic dict w/ name:	 {}
quadratic matrix:
 [[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]] 

symmetric quadratic matrix:
 [[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]] 

quadratic matrix as sparse matrix:
 


In [ ]:
%time result = mod.solve()
print("x={}".format(result.x))
print("fval={:.2f}".format(result.fval))

In [126]:
from qiskit.optimization.converters import QuadraticProgramToQubo
conv = QuadraticProgramToQubo()
mod_Qubo = conv.convert(mod)


In [127]:
print('constant:\t\t\t', mod_Qubo.objective.constant)
print('linear array as sparse matrix:\n', mod_Qubo.objective.linear.coefficients, '\n')

print('quadratic matrix as sparse matrix:\n', mod_Qubo.objective.quadratic.coefficients)

constant:			 4.3248e+27
linear array as sparse matrix:
   (0, 0)	-6.144e+25
  (0, 1)	-1.2288e+26
  (0, 2)	-2.4576e+26
  (0, 3)	-4.9152e+26
  (0, 4)	-9.8304e+26
  (0, 5)	-1.96608e+27
  (0, 6)	-3.93216e+27
  (0, 7)	-7.86432e+27
  (0, 8)	-1.572864e+28
  (0, 9)	-3.145728e+28
  (0, 10)	-6.291456e+28
  (0, 11)	-1.2582912e+29
  (0, 12)	-2.5165824e+29
  (0, 13)	-5.0331648e+29
  (0, 14)	-1.00663296e+30
  (0, 15)	-2.01326592e+30
  (0, 16)	-4.02653184e+30
  (0, 17)	-8.05306368e+30
  (0, 18)	-1.610612736e+31
  (0, 19)	-3.221225472e+31
  (0, 20)	-6.442450944e+31
  (0, 21)	-1.2884901888e+32
  (0, 22)	-2.5769803776e+32
  (0, 23)	-5.1539607552e+32
  (0, 24)	-1.03079215104e+33
  :	:
  (0, 568)	-1.3824000000000001e+26
  (0, 569)	-1.152e+25
  (0, 570)	-2.304e+25
  (0, 571)	-4.608e+25
  (0, 572)	-9.216e+25
  (0, 573)	-1.728e+26
  (0, 574)	-2.304e+25
  (0, 575)	-4.608e+25
  (0, 576)	-9.216e+25
  (0, 577)	-1.8432e+26
  (0, 578)	-3.6864e+26
  (0, 579)	-6.6816e+26
  (0, 580)	-1.728e+25
  (0, 581)	-3.456e+25
 

In [ ]:
""" it takes too long to do the next transfirmation 
op, offset = mod_Qubo.to_ising()
QPI=QuadraticProgram()
QPI.from_ising(op, offset, linear=True)
"""

In [ ]:
"""
print('constant:\t\t\t', QPI.objective.constant)
print('linear array as sparse matrix:\n', QPI.objective.linear.coefficients, '\n')
print('quadratic matrix as sparse matrix:\n', QPI.objective.quadratic.coefficients)
"""

In [ ]:
#exact, qaoa, rqaoa, grover_optimizer, admm
%time result = qaoa.solve(mod) #rqaoa, grover_optimizer run out of memory after 28min

print(result)
#statevector_simulator with 402 qubits for admm.solve(mod) and admm_rqaoa had filed after 4m

#Minimum Eigen Optimizer using VQE

In [133]:
from qiskit.circuit.library import RealAmplitudes
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.aqua.algorithms import NumPyMinimumEigensolver, VQE
from qiskit.aqua.operators import PauliExpectation, CVaRExpectation
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import LinearEqualityToPenalty
from qiskit.optimization.algorithms import MinimumEigenOptimizer
from qiskit import execute, Aer
from qiskit.aqua import aqua_globals

import numpy as np
import matplotlib.pyplot as plt
from docplex.mp.model import Model

In [134]:
aqua_globals.random_seed = 123456
n = 12            # number of assets for the variational ansatz

In [135]:
# set classical optimizer
maxiter = 100
optimizer = COBYLA(maxiter=maxiter)

# set variational ansatz
var_form = RealAmplitudes(n, reps=1)
m = var_form.num_parameters

# set backend
backend_name = 'qasm_simulator'  # use this for QASM simulator qasm_simulator or statevector_simulator
# backend_name = 'statevector_simulator'  # use this for statevector simlator
backend = Aer.get_backend(backend_name)  

# run variational optimization for different values of alpha
alphas = [1.0, 0.50, 0.25]  # confidence levels to be evaluated

In [ ]:
# dictionaries to store optimization progress and results
objectives = {alpha: [] for alpha in alphas}  # set of tested objective functions w.r.t. alpha
results = {}  # results of minimum eigensolver w.r.t alpha

# callback to store intermediate results
def callback(i, params, obj, stddev, alpha):
    # we translate the objective from the internal Ising representation
    # to the original optimization problem
    objectives[alpha] += [-(obj + offset)]  

# loop over all given alpha values
for alpha in alphas:
    
    # initialize CVaR_alpha objective
    cvar_exp = CVaRExpectation(alpha, PauliExpectation())
    cvar_exp.compute_variance = lambda x: [0]  # to be fixed in PR #1373
    
    # initialize VQE using CVaR
    vqe = VQE(expectation=cvar_exp, optimizer=optimizer, var_form=var_form, quantum_instance=backend,
              callback=lambda i, params, obj, stddev: callback(i, params, obj, stddev, alpha))
   
    # initialize optimization algorithm based on CVaR-VQE
    opt_alg = MinimumEigenOptimizer(vqe)

    # solve problem
    %time results[alpha] = opt_alg.solve(mod) #VGG qp, qubo, mod, mod_Qubo
    
    # print results
    print('alpha = {}:'.format(alpha))
    print(results[alpha])
    print()